In [396]:
import os
import h5py
import numpy as np
import pickle
import autokeras as ak
import tensorflow as tf
import pandas as pd

In [397]:
# Reading files. 

# change paths here. 

session_train=pd.read_csv('./CSV files/session_train.csv')
session_validation=pd.read_csv('./CSV files/session_val.csv')
session_test=pd.read_csv('./CSV files/final_test.csv')


train_path = './Features/TextualFeatures_csv/alltextual_withsenti_train.csv'
val_path = './Features/TextualFeatures_csv/alltextual_withsenti_valid.csv'
test_path = './Features/TextualFeatures_csv/alltextual_withsenti_test.csv'


data_validation = pd.read_csv(val_path)
data_train = pd.read_csv(train_path)
data_test = pd.read_csv(test_path)

drop_cols = ['Unnamed: 0', 'Video']

data_train.columns
data_validation.drop(drop_cols, axis = 1, inplace=True)
data_train.drop(drop_cols, axis = 1, inplace=True)
data_test.drop(drop_cols, axis = 1, inplace=True)


In [398]:
# data_train.index = 1267
# data_validation.index = 1089

# data_test.index = 489

training_name = train_path.split('/')[3].split('_')[0]+'_'+train_path.split('/')[3].split('_')[1]+'_'



In [399]:
training_name
# used later to name models and files

'alltextual_withsenti_'

In [400]:
drop_cols = ['Unnamed: 0', 'Video', 'PART.1', 'PART.2', 'GENDER', 'AGE',]
session_train.drop(drop_cols, axis = 1, inplace=True)
session_validation.drop(drop_cols, axis = 1, inplace=True)
session_test.drop(drop_cols, axis = 1, inplace=True)


In [401]:
# len(session_validation.columns)
session_test.drop(['SESSION1', 'SESSION2', 'SESSION3', 'SESSION4' ], axis = 1, inplace=True)


In [402]:
session_test.head()

,ID_y
0,8
1,56
2,66
3,67
4,86


In [403]:
data_train = pd.merge(session_train, data_train, left_on='ID_y', right_on='ID_y').drop_duplicates()
data_validation = pd.merge(session_validation, data_validation, left_on='ID_y', right_on='ID_y').drop_duplicates()
data_test = pd.merge(session_test, data_test, left_on='ID_y', right_on='ID_y').drop_duplicates()

In [404]:
data_test.head()

,ID_y,minute,session,gender,age,SentiMin,SentiMax,SentiAvg,SentiStdev,Senti0-0.20,...,TurnPercent,AvgWrdPerTurn,LongestTurn,TotalWrds,STDWrdPerTurn,TimeMin,TimeMax,TimeAvg,TimeStdev,TimeTotal
0,8,0,animals,F,41,0.001008,0.744079,0.299785,2.269071e-01,3.0,...,0.333333,3.200000,10.0,32.0,0.379473,640.0,2040.0,1188.000000,457.138929,11880.0
1,8,0,ghost,F,41,0.000057,0.497892,0.192607,1.675656e-01,8.0,...,0.441176,3.800000,7.0,57.0,0.309839,400.0,1720.0,1152.000000,304.000000,17280.0
2,8,0,lego,F,41,0.497892,0.497892,0.497892,5.551115e-17,0.0,...,0.515152,4.117647,9.0,70.0,0.028534,520.0,2720.0,1421.176471,633.513845,24160.0
3,8,0,talk,F,41,0.009673,0.632372,0.276262,2.370945e-01,3.0,...,0.318182,4.000000,8.0,28.0,0.377964,288.0,4809.0,1818.428571,1665.117401,12729.0
4,8,1,animals,F,41,0.028742,0.783591,0.400491,2.032181e-01,2.0,...,0.466667,2.142857,4.0,30.0,0.038180,400.0,2320.0,1131.428571,493.020677,15840.0


In [405]:
labels = ['OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z', 'EXTRAVERSION_Z', 'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z']


In [406]:
M_train = data_train[data_train['gender'] == 'M']
F_train = data_train[data_train['gender'] == 'F']

M_val = data_validation[data_validation['gender'] == 'M']
F_val = data_validation[data_validation['gender'] == 'F']

M_test = data_test[data_test['gender'] == 'M']
F_test = data_test[data_test['gender'] == 'F']

In [407]:
Y_train = data_train[data_train['age'] <= 30]
O_train = data_train[data_train['age'] > 30]

Y_val = data_validation[data_validation['age'] <= 30]
O_val = data_validation[data_validation['age'] > 30]

Y_test = data_test[data_test['age'] <= 30]
O_test = data_test[data_test['age'] > 30]

In [408]:
Y_train.columns

Index(['ID_y', 'OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z', 'EXTRAVERSION_Z',
       'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z', 'minute', 'session',
       'gender', 'age', 'SentiMin', 'SentiMax', 'SentiAvg', 'SentiStdev',
       'Senti0-0.20', 'Senti0.21-0.40', 'Senti0.41-0.60', 'Senti0.61-0.80',
       'Senti0.81-1.00', 'SentiTotal', 'TraitMin', 'TraitMax', 'TraitAvg',
       'TraitStdev', 'TraitTotal', 'TurnPercent', 'AvgWrdPerTurn',
       'LongestTurn', 'TotalWrds', 'STDWrdPerTurn', 'TimeMin', 'TimeMax',
       'TimeAvg', 'TimeStdev', 'TimeTotal'],
      dtype='object')

In [409]:
print(data_train[data_train['age'] <= 30].shape)
print(data_validation[data_validation['age'] <= 30].shape)
print(Y_val.shape)

(677, 35)
(870, 35)
(870, 35)


In [410]:
M_train = M_train.append(M_val)
F_train = F_train.append(F_val)

In [411]:
Y_train = Y_train.append(Y_val)
O_train = O_train.append(O_val)

In [412]:
Y_train.shape

(1547, 35)

In [413]:
df_train = data_train.append(data_validation)
df_validation = data_validation
df_test = data_test

In [414]:
# Combined 
train_O = df_train.pop(labels[0]).to_numpy()
train_C = df_train.pop(labels[1]).to_numpy()
train_E = df_train.pop(labels[2]).to_numpy()
train_A = df_train.pop(labels[3]).to_numpy()
train_N = df_train.pop(labels[4]).to_numpy()

val_O = df_validation.pop(labels[0]).to_numpy()
val_C = df_validation.pop(labels[1]).to_numpy()
val_E = df_validation.pop(labels[2]).to_numpy()
val_A = df_validation.pop(labels[3]).to_numpy()
val_N = df_validation.pop(labels[4]).to_numpy()

# male train OCEAN 
M_train_O = M_train.pop(labels[0]).to_numpy()
M_train_C = M_train.pop(labels[1]).to_numpy()
M_train_E = M_train.pop(labels[2]).to_numpy()
M_train_A = M_train.pop(labels[3]).to_numpy()
M_train_N= M_train.pop(labels[4]).to_numpy()

# female train OCEAN
F_train_O = F_train.pop(labels[0]).to_numpy()
F_train_C = F_train.pop(labels[1]).to_numpy()
F_train_E = F_train.pop(labels[2]).to_numpy()
F_train_A = F_train.pop(labels[3]).to_numpy()
F_train_N= F_train.pop(labels[4]).to_numpy()

# male validation OCEAN

M_val_O = M_val.pop(labels[0]).to_numpy()
M_val_C = M_val.pop(labels[1]).to_numpy()
M_val_E = M_val.pop(labels[2]).to_numpy()
M_val_A = M_val.pop(labels[3]).to_numpy()
M_val_N = M_val.pop(labels[4]).to_numpy()

# female validation OCEAN

F_val_O = F_val.pop(labels[0]).to_numpy()
F_val_C = F_val.pop(labels[1]).to_numpy()
F_val_E = F_val.pop(labels[2]).to_numpy()
F_val_A = F_val.pop(labels[3]).to_numpy()
F_val_N = F_val.pop(labels[4]).to_numpy()

In [415]:


# male train OCEAN 
Y_train_O = Y_train.pop(labels[0]).to_numpy()
Y_train_C = Y_train.pop(labels[1]).to_numpy()
Y_train_E = Y_train.pop(labels[2]).to_numpy()
Y_train_A = Y_train.pop(labels[3]).to_numpy()
Y_train_N=  Y_train.pop(labels[4]).to_numpy()

# female train OCEAN
O_train_O = O_train.pop(labels[0]).to_numpy()
O_train_C = O_train.pop(labels[1]).to_numpy()
O_train_E = O_train.pop(labels[2]).to_numpy()
O_train_A = O_train.pop(labels[3]).to_numpy()
O_train_N=  O_train.pop(labels[4]).to_numpy()

# male validation OCEAN

Y_val_O = Y_val.pop(labels[0]).to_numpy()
Y_val_C = Y_val.pop(labels[1]).to_numpy()
Y_val_E = Y_val.pop(labels[2]).to_numpy()
Y_val_A = Y_val.pop(labels[3]).to_numpy()
Y_val_N = Y_val.pop(labels[4]).to_numpy()

# female validation OCEAN

O_val_O = O_val.pop(labels[0]).to_numpy()
O_val_C = O_val.pop(labels[1]).to_numpy()
O_val_E = O_val.pop(labels[2]).to_numpy()
O_val_A = O_val.pop(labels[3]).to_numpy()
O_val_N = O_val.pop(labels[4]).to_numpy()

In [416]:
data_cols = [ 'ID_y', 'minute']
drop_cols = []

train_data = df_train[data_cols]
val_data   = df_validation[data_cols]
test_data  = df_test[data_cols]

Y_train_data = Y_train[data_cols]
Y_val_data   = Y_val[data_cols]
Y_test_data  = Y_test[data_cols]

O_train_data = O_train[data_cols]
O_val_data   = O_val[data_cols]
O_test_data  = O_test[data_cols]

M_train_data = M_train[data_cols]
F_train_data = F_train[data_cols]
M_val_data = M_val[data_cols]
F_val_data = F_val[data_cols]
M_test_data = M_test[data_cols]
F_test_data = F_test[data_cols]

In [417]:
Y_train_data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,   48,   49,
            ...
            2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160],
           dtype='int64', length=1547)

In [418]:
data_cols = ['ID_y','minute', 'session', 'gender', 'age']
# combined
train_com = df_train.drop(data_cols+drop_cols, axis = 1)
val_com   = df_validation.drop(data_cols+drop_cols, axis = 1)
test_com  = df_test.drop(data_cols+drop_cols, axis = 1)



# Age
Y_train = Y_train.drop(data_cols+drop_cols, axis = 1)
Y_val   = Y_val.drop(data_cols+drop_cols, axis = 1)
Y_test  = Y_test.drop(data_cols+drop_cols, axis = 1)

O_train = O_train.drop(data_cols+drop_cols, axis = 1)
O_val   = O_val.drop(data_cols+drop_cols, axis = 1)
O_test  = O_test.drop(data_cols+drop_cols, axis = 1)



# Gender
M_train = M_train.drop(data_cols+drop_cols, axis = 1)
M_val = M_val.drop(data_cols+drop_cols, axis = 1)
M_test = M_test.drop(data_cols+drop_cols, axis = 1)

F_train = F_train.drop(data_cols+drop_cols, axis = 1)
F_val = F_val.drop(data_cols+drop_cols, axis = 1)
F_test = F_test.drop(data_cols+drop_cols, axis = 1)



In [419]:
# Comvined
train_data.reset_index(drop = True, inplace = True)
val_data.reset_index(drop = True, inplace = True)
test_data.reset_index(drop = True, inplace = True)


# Age

Y_train.reset_index(drop=True, inplace=True)
Y_train_data.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
Y_test_data.reset_index(drop = True, inplace = True)
Y_val.reset_index(drop=True, inplace=True)
Y_val_data.reset_index(drop=True, inplace=True)

O_train.reset_index(drop=True, inplace=True)
O_train_data.reset_index(drop=True, inplace=True)
O_val.reset_index(drop=True, inplace=True)
O_val_data.reset_index(drop=True, inplace=True)
O_test.reset_index(drop=True, inplace=True)
O_test_data.reset_index(drop = True, inplace = True)

# Gender
M_train.reset_index(drop=True, inplace=True)
M_train_data.reset_index(drop=True, inplace=True)
M_test.reset_index(drop=True, inplace=True)
M_test_data.reset_index(drop = True, inplace = True)
M_val.reset_index(drop=True, inplace=True)
M_val_data.reset_index(drop=True, inplace=True)

F_train.reset_index(drop=True, inplace=True)
F_train_data.reset_index(drop=True, inplace=True)
F_val.reset_index(drop=True, inplace=True)
F_val_data.reset_index(drop=True, inplace=True)
F_test.reset_index(drop=True, inplace=True)
F_test_data.reset_index(drop = True, inplace = True)

In [420]:
train_data.shape

(2356, 2)

In [421]:
train_C.shape

(2356,)

In [422]:

OCEAN_models = ['Model_O', 'Model_C', 'Model_E', 'Model_A', 'Model_N']
# Combined
train_com_np = np.array(train_com)
val_com_np = np.array(val_com)
test_com_np = np.array(test_com)

# Age
Y_train_np = np.array(Y_train)
Y_test_np  = np.array(Y_test)
Y_val_np   = np.array(Y_val)


O_train_np = np.array(O_train)
O_val_np   = np.array(O_val)
O_test_np  = np.array(O_test)

# GEnder
M_train_np = np.array(M_train)
M_test_np  = np.array(M_test)
M_val_np   = np.array(M_val)


F_train_np = np.array(F_train)
F_val_np   = np.array(F_val)
F_test_np  = np.array(F_test)



In [423]:
# Combined
comb_O_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_O))
comb_C_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_C))
comb_E_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_E))
comb_A_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_A))
comb_N_train_set = tf.data.Dataset.from_tensor_slices((train_com_np, train_N))

comb_O_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_O))
comb_C_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_C))
comb_E_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_E))
comb_A_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_A))
comb_N_validation_set = tf.data.Dataset.from_tensor_slices((val_com_np, val_N))


# Age

youngO_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_O))
youngC_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_C))
youngE_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_E))
youngA_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_A))
youngN_train_set      = tf.data.Dataset.from_tensor_slices((Y_train_np, Y_train_N))
youngO_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_O))
youngC_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_C))
youngE_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_E))
youngA_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_A))
youngN_validation_set = tf.data.Dataset.from_tensor_slices((Y_val_np, Y_val_N))

oldO_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_O))
oldC_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_C))
oldE_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_E))
oldA_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_A))
oldN_validation_set = tf.data.Dataset.from_tensor_slices((O_val_np, O_val_N))
oldO_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_O))
oldC_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_C))
oldE_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_E))
oldA_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_A))
oldN_train_set      = tf.data.Dataset.from_tensor_slices((O_train_np, O_train_N))


# Gender
maleO_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_O))
maleC_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_C))
maleE_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_E))
maleA_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_A))
maleN_train_set = tf.data.Dataset.from_tensor_slices((M_train_np, M_train_N))

femaleO_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_O))
femaleC_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_C))
femaleE_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_E))
femaleA_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_A))
femaleN_train_set = tf.data.Dataset.from_tensor_slices((F_train_np, F_train_N))

maleO_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_O))
maleC_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_C))
maleE_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_E))
maleA_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_A))
maleN_validation_set = tf.data.Dataset.from_tensor_slices((M_val_np, M_val_N))

femaleO_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_O))
femaleC_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_C))
femaleE_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_E))
femaleA_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_A))
femaleN_validation_set = tf.data.Dataset.from_tensor_slices((F_val_np, F_val_N))


In [424]:
# combined
comb_train_set = [comb_O_train_set, 
                  comb_C_train_set, 
                  comb_E_train_set, 
                  comb_A_train_set, 
                  comb_N_train_set]

comb_validation_set = [comb_O_validation_set, 
                       comb_C_validation_set, 
                       comb_E_validation_set, 
                       comb_A_validation_set, 
                       comb_N_validation_set]

# Age
young_train_set =     [youngO_train_set, 
                       youngC_train_set, 
                       youngE_train_set, 
                       youngA_train_set, 
                       youngN_train_set]
young_validation_set =[youngO_validation_set, 
                       youngC_validation_set, 
                       youngE_validation_set, 
                       youngA_validation_set, 
                       youngN_validation_set]

old_train_set =         [oldO_train_set, 
                         oldC_train_set, 
                         oldE_train_set,
                         oldA_train_set,
                         oldN_train_set]
old_validation_set =    [oldO_validation_set,
                         oldC_validation_set,
                         oldE_validation_set,
                         oldA_validation_set,
                         oldN_validation_set]

# Gender
male_train_set =      [maleO_train_set, 
                       maleC_train_set, 
                       maleE_train_set, 
                       maleA_train_set, 
                       maleN_train_set]
male_validation_set = [maleO_validation_set, 
                       maleC_validation_set, 
                       maleE_validation_set, 
                       maleA_validation_set, 
                       maleN_validation_set]

female_train_set =      [femaleO_train_set, 
                         femaleC_train_set, 
                         femaleE_train_set,
                         femaleA_train_set,
                         femaleN_train_set]
female_validation_set = [femaleO_validation_set,
                         femaleC_validation_set,
                         femaleE_validation_set,
                         femaleA_validation_set,
                         femaleN_validation_set]

model_names = ['Model_O','Model_C','Model_E','Model_A','Model_N']


In [425]:
training_name

'alltextual_withsenti_'

# Prediction


In [426]:

# Change this according to the path of male and female
male_path = './Textual Models/'+training_name+'/Gender/Male/'
female_path = './Textual Models/'+training_name+'/Gender/Female/'

young_path = './Textual Models/'+training_name+'/Age/Young/'
old_path = './Textual Models/'+training_name+'/Age/Old/'

separation = male_path.split('/')[3]
print(separation)

# change this to whatever you have named the folders where each model is inside male and female
OCEAN_models = ['Model_O', 'Model_C', 'Model_E', 'Model_A', 'Model_N']
# Make sure they have similar names
for mod in OCEAN_models:
    
    model = tf.keras.models.load_model(male_path+mod)
    prediction = model.predict(M_test_np)
    # print((prediction))
    print(len(prediction))
    prediction = np.array(prediction)
    print(prediction.shape)
    M_test_data = pd.concat([M_test_data, pd.DataFrame(prediction)], axis=1)
# Here F_test_data is only the ID and minute columns of the test data you're passing in, which we are merging the 
# predictions with. 
# Make sure you change it to whatever your dataframe is.

# F_test_data.to_csv("./Results/fb_pred.csv")

"""# Making predictions based on female models for OCEAN individually"""

for mod in OCEAN_models:
    model = tf.keras.models.load_model(female_path+mod)
    prediction = model.predict(F_test_np)
    print(len(prediction))
    prediction = np.array(prediction)
    F_test_data = pd.concat([F_test_data, pd.DataFrame(prediction)], axis=1)

total_test_data = F_test_data.append(M_test_data)
print(total_test_data.columns)
total_test_data=total_test_data.sort_values(by=['ID_y'])
total_test_data.to_csv("./Results/"+training_name+"pred_"+separation+"_minute.csv")
print("Prediction over")

Gender
285
(285, 1)
285
(285, 1)
285
(285, 1)
285
(285, 1)
285
(285, 1)
204
204
204
204
204
Index(['ID_y', 'minute', 0, 0, 0, 0, 0], dtype='object')
Prediction over


In [427]:
total_test_data.head()

,ID_y,minute,0,0,0,0,0
0,8,0,0.212068,0.270076,-0.282435,0.015398,-0.217193
22,8,6,1.094202,0.257162,-0.086997,0.276457,-0.045503
21,8,5,-0.061600,0.047626,-0.089956,-0.182099,-0.074689
20,8,5,-0.045719,0.055427,-0.183186,-0.120476,0.009236
19,8,4,0.779887,0.212846,0.021240,0.028920,-0.147573


In [395]:
total_test_data.drop(['minute'], axis = 1, inplace=True)
df_fb = total_test_data.reset_index(drop= True)

mean_df_fb = df_fb.groupby(['ID_y'], as_index=False).mean()
mean_df_fb.columns = [0, 1, 2, 3, 4, 5]
mean_df_fb.to_csv("./Results/"+training_name+"pred_"+separation+"_mean.csv")